<a href="https://colab.research.google.com/github/CMallart/ateliers-NN/blob/main/TP1_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
from sklearn.datasets import load_wine
from sklearn.model_selection import KFold

# Useful functions

In [ ]:
def one_hot_encode_database(database, list_id):
  """One hot encode the columns of the database specified in list_id

  return a copy of the database, on hot encoded
  """
  encoded_database = np.empty(shape=(database.shape[0], 0), dtype=float)
  for id in range(database.shape[1]):
      if id in list_id:
          original_column = database[:, id]
          encoded_column = to_categorical(original_column.astype(int), num_classes=int(np.max(original_column)+1))
          encoded_database = np.column_stack((encoded_database, encoded_column))
      else:
          original_column = database[:, id]
          encoded_database = np.column_stack((encoded_database, original_column))
  return encoded_database

In [ ]:
def normalize_database(database, list_id):
  """Normalize between 0 and 1 each column of the database specified in list_id.

  return a copy of the database, normalized. 
  """
  encoded_database = database.copy()
  for id in list_id:
      encoded_database[:, id] = (encoded_database[:, id] - np.amin(encoded_database[:, id])) / (np.amax(encoded_database[:, id]) - np.amin(encoded_database[:, id]))
  return encoded_database

In [ ]:
def build_NN(layer_list, input_dim, output_dim, lr=0.001):
  """Construct a layer composed of dense layers, which dimensions are definded in the layer_list argument.

  return the constructed and compiled model.
  """
  # Q7: add layers and "compile" the model
  model = Sequential()
  #Add layers
  model.add(Dense(layer_list[0], input_dim=input_dim, activation='relu'))
  for layer in layer_list[1:]:
      model.add(Dense(layer, activation='relu'))
  model.add(Dense(output_dim))

  #Compile the network
  model.compile(loss=CategoricalCrossentropy(from_logits=True), optimizer=Adam(lr), metrics=['accuracy'])
  return model

# Main

## Hyper-parameters

In [ ]:
nb_fold = 5
layer_list = [50, 50, 20]

## Initialization

In [ ]:
rng = np.random.default_rng()

## Prepare data

In [ ]:

database = load_wine()
print('--- original database ---\n')
print(f'{database.data.shape[0]} examples, {database.data.shape[1]} features')
for i in [10, 80, 140]:
  print(f'label of example {i:3d}: {database.target[i]}')

# Q7: one hot encode the class and normalize the features of the database here
data = normalize_database(database.data, range(database.data.shape[1]))
target = one_hot_encode_database(database.target.reshape([-1,1]), [0])

random_indices = np.arange(data.shape[0])
rng.shuffle(random_indices)
data = data[random_indices,:]
target = target[random_indices,:]
print('\n--- after randomization ---\n')
print(f'{data.shape[0]} examples, {data.shape[1]} features')
for i in [10, 80, 140]:
  print(f'label of example {i:3d}: {target[i]}')

--- original database ---

178 examples, 13 features
label of example  10: 0
label of example  80: 1
label of example 140: 2

--- after randomization ---

178 examples, 13 features
label of example  10: [1. 0. 0.]
label of example  80: [1. 0. 0.]
label of example 140: [0. 0. 1.]


## Build / Fit / avaluate (quick check) 

In [ ]:
# Q10: quick check
model = build_NN(layer_list, data.shape[1], target.shape[1], lr=0.1)
model.summary()
model.fit(data, target, batch_size=16, epochs=100)
model.evaluate(data, target)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                700       
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_2 (Dense)              (None, 20)                1020      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 63        
Total params: 4,333
Trainable params: 4,333
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
12/12 [==============================] - 0s 2ms/step - loss: 1.1218 - accuracy: 0.4719
Epoch 2/100
12/12 [==============================] - 0s 1ms/step - loss: 0.7480 - accuracy: 0.6461
Epoch 3/100
12/12 [==============================] - 0

[1.0053917321783956e-05, 1.0]

## Build / Fit / avaluate (cross-validated) 

In [ ]:
gt_and_pred = np.empty(shape=(2, 0))

for train_index, test_index in KFold(n_splits=nb_fold).split(data):
  training_data, validation_data = data[train_index], data[test_index]
  training_target, validation_target = target[train_index], target[test_index]

  # Q11: test several learning rates
  model = build_NN(layer_list, data.shape[1], target.shape[1], lr=0.1)

  callback = EarlyStopping(patience=5)
  # Q13: add EarlyStopping() callback
  model.fit(training_data, training_target, batch_size=16, epochs=200, validation_data=(validation_data, validation_target), shuffle=True, callbacks=[callback])

  fold_prediction = model.predict(validation_data)
  gt_and_pred = np.column_stack((gt_and_pred, np.array([np.argmax(validation_target, axis=1), np.argmax(fold_prediction, axis=1)])))

Epoch 1/200
9/9 [==============================] - 0s 18ms/step - loss: 1.2656 - accuracy: 0.4648 - val_loss: 0.8046 - val_accuracy: 0.5000
Epoch 2/200
9/9 [==============================] - 0s 3ms/step - loss: 0.6600 - accuracy: 0.6127 - val_loss: 0.5386 - val_accuracy: 0.6944
Epoch 3/200
9/9 [==============================] - 0s 3ms/step - loss: 0.5807 - accuracy: 0.6127 - val_loss: 0.5161 - val_accuracy: 0.6667
Epoch 4/200
9/9 [==============================] - 0s 3ms/step - loss: 0.4668 - accuracy: 0.7324 - val_loss: 0.3242 - val_accuracy: 0.8056
Epoch 5/200
9/9 [==============================] - 0s 3ms/step - loss: 0.4114 - accuracy: 0.8380 - val_loss: 0.6410 - val_accuracy: 0.7500
Epoch 6/200
9/9 [==============================] - 0s 3ms/step - loss: 0.3337 - accuracy: 0.8732 - val_loss: 0.2309 - val_accuracy: 0.8611
Epoch 7/200
9/9 [==============================] - 0s 3ms/step - loss: 0.1586 - accuracy: 0.9155 - val_loss: 0.2410 - val_accuracy: 0.9444
Epoch 8/200
9/9 [=========

In [ ]:
# Q12: display confusion matrix and balanced accuracy here
acc = balanced_accuracy_score(gt_and_pred[0], gt_and_pred[1])
conf_mat = confusion_matrix(gt_and_pred[0], gt_and_pred[1], labels=[0, 1, 2])
print("balanced_accuracy=", acc)
print(conf_mat)